In [1]:
import numpy as np
import pandas as pd
!pip install xlrd

In [2]:
toronto_data = pd.read_excel (r'toronto.xlsx') #read into the dataset
toronto_data = toronto_data[toronto_data.Borough != 'Not assigned'] #drop columns with Borough "Not assigned"
toronto_data["Neighborhood"] = toronto_data["Neighborhood"].str.replace("/", ",") #replace / with ,
toronto_data.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [3]:
toronto_data.shape

(103, 3)

In [ ]:
!conda install -c conda-forge geopy --yes 
!pip install geocoder
import geocoder # import geocoder

In [5]:
longitude = []
latitude = []
for postal_code in range(toronto_data['Postal code'].count()):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g =  geocoder.arcgis('{}, Toronto, Ontario'.format(toronto_data.iloc[postal_code, 0]))
        lat_lng_coords = g.latlng
        
        latitude.append(lat_lng_coords[0])
        longitude.append(lat_lng_coords[1])

In [6]:
latitude = pd.DataFrame(latitude, columns=['Latitude'])
longitude = pd.DataFrame(longitude, columns=['Longitude'])
result = pd.concat([latitude, longitude], axis=1, sort=False)
toronto_data.reset_index(drop=True, inplace=True)
sorted_data = pd.concat([toronto_data, result], axis=1, sort=False)
sorted_data

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653340,-79.509766
99,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
100,M7Y,East Toronto,Business reply mail Processing Centre,43.648700,-79.385450
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.632798,-79.493017


In [8]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

Solving environment: done

# All requested packages already installed.



In [9]:
toronto_borough = sorted_data[sorted_data['Borough'].str.contains('Toronto')]

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.6534817, -79.3839347.


In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_borough['Latitude'], toronto_borough['Longitude'], toronto_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
CLIENT_ID = 'ZRYMX3YW2SO3FPPPJUDPDM1BYUYABH1W0PCEPIRR5P0LYXI2' # your Foursquare ID
CLIENT_SECRET = 'ICDF3XDHHUFHPC4K1J5FMEUZFLMFWXDO0XJGX2HT4GPGE1V4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
import requests
toronto_venues = getNearbyVenues(names=toronto_borough['Neighborhood'],
                                   latitudes=toronto_borough['Latitude'],
                                   longitudes=toronto_borough['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District , Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosed

In [37]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,9,9,9,9,9,9
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,30,30,30,30,30,30
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",15,15,15,15,15,15
Central Bay Street,12,12,12,12,12,12
Christie,3,3,3,3,3,3
Church and Wellesley,30,30,30,30,30,30
"Commerce Court , Victoria Hotel",30,30,30,30,30,30
Davisville,17,17,17,17,17,17


In [38]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 155 uniques categories.
